In [103]:
import pandas as pd
import numpy as np
import tensorflow
import keras

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [104]:
data = pd.read_csv('/Users/aaronmichaelbroderick/Desktop/Data Science/Data/devanagari.csv')

## Cleaning data

In [87]:
Y = data['character']

columns = list(data)
columns.remove('character')
X = data[columns]

In [88]:
# Change strings into numerical to work well with Keras

letters = list(set(Y))
Y_test = list(Y)

for i in range(len(letters)):
    for j in range(len(Y_test)):
        if letters[i] == Y_test[j]:
            Y_test[j] = i

In [89]:
results = pd.DataFrame()
results['cat'] = Y
results['num'] = Y_test

Y = results.num

## Make the model

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
x_trainD, x_testD, y_trainD, y_testD = train_test_split(X,Y, test_size = 0.2)

In [93]:
# Training parameters.
batch_size = 64
num_classes = 46
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

In [95]:
# Changes the data from a pandas dataframe to a numpy array
x_trainD = x_trainD.values
x_testD = x_testD.values

In [96]:
# Reshapes data to 4D for Hierarchical RNN.
x_trainD = x_trainD.reshape(x_trainD.shape[0], 32, 32, 1)
x_testD = x_testD.reshape(x_testD.shape[0], 32, 32, 1)
x_trainD = x_trainD.astype('float32')
x_testD = x_testD.astype('float32')
x_trainD /= 255
x_testD /= 255
print('x_trainD shape:', x_trainD.shape)
print(x_trainD.shape[0], 'train samples')
print(x_testD.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_trainD = keras.utils.to_categorical(y_trainD, num_classes)
y_testD = keras.utils.to_categorical(y_testD, num_classes)

x_trainD shape: (73600, 32, 32, 1)
73600 train samples
18400 test samples


In [98]:
row, col, pixel = x_trainD.shape[1:]
x = Input(shape=(row, col, pixel))
x

<tf.Tensor 'input_3:0' shape=(?, 32, 32, 1) dtype=float32>

In [99]:
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)
encoded_rows

<tf.Tensor 'time_distributed_3/Reshape_1:0' shape=(?, 32, 32) dtype=float32>

In [100]:
encoded_columns = LSTM(col_hidden)(encoded_rows)
encoded_columns

<tf.Tensor 'lstm_6/TensorArrayReadV3:0' shape=(?, 32) dtype=float32>

In [101]:
y_testD.shape

(18400, 46)

In [102]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_trainD, y_trainD,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_testD, y_testD))

# Evaluation.
scores = model.evaluate(x_testD, y_testD, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 73600 samples, validate on 18400 samples
Epoch 1/3
73600/73600 [==============================] - 149s 2ms/step - loss: 2.9287 - acc: 0.2180 - val_loss: 2.2810 - val_acc: 0.3704
Epoch 2/3
73600/73600 [==============================] - 149s 2ms/step - loss: 1.7789 - acc: 0.4951 - val_loss: 1.4156 - val_acc: 0.6042
Epoch 3/3
73600/73600 [==============================] - 148s 2ms/step - loss: 1.1507 - acc: 0.6703 - val_loss: 0.9778 - val_acc: 0.7188
Test loss: 0.9778406575451727
Test accuracy: 0.7188043478260869
